In [4]:
from jupyter_client.session import extract_header
from openai import OpenAI
import os
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

client = OpenAI(api_key=os.getenv("OPENAI_API_KEU"))
# 1print("\n" + response.output_text)

In [37]:
def get_completions_functional(model="gpt-4", **kwargs):

    return client.responses.create(model=model, **kwargs)

In [108]:
# . Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]


def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."

In [109]:
input_list = [{"role": "user", "content": "What is my horoscope? I am an Aquarius."}]

In [110]:
# Create a running input list we will add to over time

# 2. Prompt the model with tools defined

response = get_completions_functional(
    model="gpt-4", tools=tools, temperature=0, input=input_list
)
# Save function call outputs for subsequent requests

In [111]:
from IPython.display import display, Markdown

list(zip(response.model_dump().keys(), response.model_dump().values()))

[('id', 'resp_01fde4fbfe1d8b0c00694ff36787fc81a0833fa63626958b81'),
 ('created_at', 1766847336.0),
 ('error', None),
 ('incomplete_details', None),
 ('instructions', None),
 ('metadata', {}),
 ('model', 'gpt-4-0613'),
 ('object', 'response'),
 ('output',
  [{'arguments': '{\n  "sign": "Aquarius"\n}',
    'call_id': 'call_CgOnTN8li0p4ePm6KcXfGvqK',
    'name': 'get_horoscope',
    'type': 'function_call',
    'id': 'fc_01fde4fbfe1d8b0c00694ff36a556081a08cee8eda684df409',
    'status': 'completed'}]),
 ('parallel_tool_calls', True),
 ('temperature', 0.0),
 ('tool_choice', 'auto'),
 ('tools',
  [{'name': 'get_horoscope',
    'parameters': {'type': 'object',
     'properties': {'sign': {'type': 'string',
       'description': 'An astrological sign like Taurus or Aquarius'}},
     'required': ['sign'],
     'additionalProperties': False},
    'strict': True,
    'type': 'function',
    'description': "Get today's horoscope for an astrological sign."}]),
 ('top_p', 1.0),
 ('background', Fals

In [112]:
response.output

[ResponseFunctionToolCall(arguments='{\n  "sign": "Aquarius"\n}', call_id='call_CgOnTN8li0p4ePm6KcXfGvqK', name='get_horoscope', type='function_call', id='fc_01fde4fbfe1d8b0c00694ff36a556081a08cee8eda684df409', status='completed')]

In [113]:
input_list += response.output

In [114]:
input_list

[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'},
 ResponseFunctionToolCall(arguments='{\n  "sign": "Aquarius"\n}', call_id='call_CgOnTN8li0p4ePm6KcXfGvqK', name='get_horoscope', type='function_call', id='fc_01fde4fbfe1d8b0c00694ff36a556081a08cee8eda684df409', status='completed')]

In [115]:
x = response.output[0].model_dump()

In [116]:
x

{'arguments': '{\n  "sign": "Aquarius"\n}',
 'call_id': 'call_CgOnTN8li0p4ePm6KcXfGvqK',
 'name': 'get_horoscope',
 'type': 'function_call',
 'id': 'fc_01fde4fbfe1d8b0c00694ff36a556081a08cee8eda684df409',
 'status': 'completed'}

In [117]:
x["name"] += get_horoscope(x["arguments"])

In [118]:
x["name"]

'get_horoscope{\n  "sign": "Aquarius"\n}: Next Tuesday you will befriend a baby otter.'

In [119]:
import json

horoscope = json.dumps(get_horoscope(json.loads(x["arguments"])))

input_list.append(
    {"type": "function_call_output", "call_id": x["call_id"], "output": horoscope}
)

In [30]:
# import json
# for item in response.output:
#     if item.type == "function_call":
#         if item.name == "get_horoscope":
#             # 3. Execute the function logic for get_horoscope
#             horoscope = get_horoscope(json.loads(item.arguments))
#             print(horoscope)
#             # 4. Provide function call results to the model
#             input_list.append({
#                 "type": "function_call_output",
#                 "call_id": item.call_id,
#                 "output": json.dumps({
#                   "horoscope": horoscope
#                 })
#             })

{'sign': 'Aquarius'}: Next Tuesday you will befriend a baby otter.


In [120]:
print("Final input:")
print(input_list)

Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseFunctionToolCall(arguments='{\n  "sign": "Aquarius"\n}', call_id='call_CgOnTN8li0p4ePm6KcXfGvqK', name='get_horoscope', type='function_call', id='fc_01fde4fbfe1d8b0c00694ff36a556081a08cee8eda684df409', status='completed'), {'type': 'function_call_output', 'call_id': 'call_CgOnTN8li0p4ePm6KcXfGvqK', 'output': '"{\'sign\': \'Aquarius\'}: Next Tuesday you will befriend a baby otter."'}]


In [121]:
response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!

In [122]:
print("Final output:")
print(response.model_dump_json(indent=2))

Final output:
{
  "id": "resp_01fde4fbfe1d8b0c00694ff396442481a0b1bc91a26700a75e",
  "created_at": 1766847382.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_01fde4fbfe1d8b0c00694ff397e58481a0959d13fc59e1747b",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_01fde4fbfe1d8b0c00694ff3a149b881a0891a6b651da36f41",
      "content": [
        {
          "annotations": [],
          "text": "{'sign': 'Aquarius'}: Next Tuesday you will befriend a baby otter.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "too

In [123]:
print(len(response.output))

2


In [124]:
response.output[1].model_dump()

{'id': 'msg_01fde4fbfe1d8b0c00694ff3a149b881a0891a6b651da36f41',
 'content': [{'annotations': [],
   'text': "{'sign': 'Aquarius'}: Next Tuesday you will befriend a baby otter.",
   'type': 'output_text',
   'logprobs': []}],
 'role': 'assistant',
 'status': 'completed',
 'type': 'message'}

In [125]:
response.output_text

"{'sign': 'Aquarius'}: Next Tuesday you will befriend a baby otter."

In [3]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model="gpt-4"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [1]:
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Retrieves current weather for the given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia",
                },
                "units": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Units the temperature will be returned in.",
                },
            },
            "required": ["location", "units"],
            "additionalProperties": False,
        },
        "strict": True,
    }
]

In [11]:
from pydantic import BaseModel, Field


class GetCurrentWeather(BaseModel):
    """
    Get the current weather for a specific location.
    The user can ask for any location, so infer the location from the user's prompt.
    """

    location: str = Field(description="The city and state, e.g. San Francisco, CA")
    unit: str = Field(
        description="The temperature unit to use. Infer this from the user's location, default is 'fahrenheit'",
        default="fahrenheit",
    )

In [14]:
prompt = [{"role": "user", "content": "Give weather for Noida, India"}]
response = client.responses.parse(model="gpt-4", text_format=GetCurrentWeather)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: 'text.format' of type 'json_schema' is not supported with model version `gpt-4`.", 'type': 'invalid_request_error', 'param': 'text.format', 'code': None}}

In [9]:
content = response.output_parsed

In [10]:
content

GetWeather(location='Noida, India', units='metric')

In [ ]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)
    if name == "send_email":
        return send_email(**args)

In [ ]:
for tool_call in response.output:
    if tool_call.type != "function_call":
        continue

    name = tool_call.name
    args = json.loads(tool_call.arguments)

    result = call_function(name, args)
    input_messages.append(
        {
            "type": "function_call_output",
            "call_id": tool_call.call_id,
            "output": str(result),
        }
    )